In [2]:
import numpy as np
from matplotlib import figure, gridspec,colors

import scanner_interpretation as scani

import healpy as hp

from matplotlib import colors,cm
import matplotlib.pyplot as plt
import matplotlib as mpl
from astropy import visualization as vis

import dill

from gammapy.datasets import Datasets
from gammapy.data import DataStore,ObservationFilter

from pathlib import Path
from astropy.coordinates import SkyCoord, Angle
from regions import CircleSkyRegion, Regions



with open("pkl_jugs/mplets.pkl","rb") as f:
    mplets = dill.load(f)

nosource_mask = mplets.table["TEVCAT_DISTANCES_DEG"] >= 0.2
Nmin4_mask = mplets.table["Nmax"] >= 4
Nmin5_mask = mplets.table["Nmax"] >= 5
dt1sec_mask = mplets.table["dt"] <= 1e9
xgal_mask = np.abs(mplets.table["MEDIAN_GLAT"]) > 5

Get sphere distances for one observation

In [3]:
mymask = xgal_mask * nosource_mask
reduced = mplets.table[mymask]
obsids = reduced["OBS_ID"]
print(len(obsids))

25782


In [8]:

current_obs_index = 0

hess1_datastore = DataStore.from_dir("$HESS1")
hess1u_datastore = DataStore.from_dir("$HESS1U")
myobs = hess1_datastore.get_observations([obsids[current_obs_index]])
# print(np.unique([obs.obs_info["RADECSYS"] for obs in myobs]))
phottable = myobs[0].events.table

run_dist = scani.sphere_dist(phottable["RA"].data, phottable["DEC"].data, reduced[current_obs_index]["MEDIAN_RA"], reduced[current_obs_index]["MEDIAN_DEC"])

In [10]:
phottable[:2]

EVENT_ID,EVENT_ID_HESS,BUNCH_ID_HESS,OBS_ID,TIME,MULTIP,RA,DEC,DIR_ERR,SKYX_RADEC,SKYY_RADEC,DETX,DETY,FOV_ALTAZ_THETA,FOV_ALTAZ_PHI,FOV_ALTAZ_LON,FOV_ALTAZ_LAT,FOV_RADEC_THETA,FOV_RADEC_PHI,FOV_RADEC_LON,FOV_RADEC_LAT,ALT,AZ,COREX,COREY,CORE_ERR,XMAX,XMAX_ERR,ENERGY,ENERGY_ERR,HIL_MSW,HIL_MSW_ERR,HIL_MSL,HIL_MSL_ERR
,,,,s,,deg,deg,deg,deg,deg,deg,deg,deg,deg,deg,deg,deg,deg,deg,deg,deg,deg,m,m,m,m,m,TeV,TeV,,,,
int64,uint32,uint32,uint32,float64,int16,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32
120259084333,45,28,18361,96157131.04546523,2,84.15269,23.469427,0.061122123,-0.95593166,0.47646198,-0.888458,-0.5927528,1.0680287,-33.712044,-0.888458,-0.5927528,1.0679687,116.4929,0.47645098,0.95580995,37.471977,331.1664,-143.24669,-81.35084,1e-45,172.71237,1.0579411e-07,0.67420715,1.0,-1.0197905,0.26451847,-0.7061351,0.35866004
120259084358,70,28,18361,96157131.22693157,2,83.71886,21.537926,0.0,0.9765909,0.07956559,0.4162463,0.88698065,0.97978663,115.137985,0.4162463,0.88698065,0.9797312,-94.65776,0.07956554,-0.9764953,38.955517,329.51102,751.4099,-405.82053,1e-45,106.964645,8.6040913e-10,9.423786,1.0,-1.0877063,0.21198279,-0.45175344,0.30176064


In [ ]:
myhist = vis.hist(run_dist,histtype="step",bins="freedman")
plt.xlabel("Angular distance [deg]")
plt.ylabel("Photon counts")
plt.title(f"Angular distance from median RADEC of multiplet to photons in run {obsids[current_obs_index]}")
ax = plt.gca()
ylims = ax.get_ylim()
plt.vlines(5,ylims[0],ylims[1],color="red",ls="--",label="5 deg")
ax.set_ylim(*ylims)
ax.set_xlim(-.5,7)
plt.legend()
# plt.yscale("log")

In [ ]:
myobs[0].obs_info

In [ ]:
mask = run_dist < 0.1
photoncount = len(phottable[mask][:])#/myobs[0].obs_info["LIVETIME"]/60
print(photoncount)
plt.scatter(phottable[mask]["TIME"],[1 for i in range(photoncount)],s=4,marker="x")
plt.xlim(myobs[0].obs_info["TSTART"],myobs[0].obs_info["TSTOP"])
mplet_start_time_ns = reduced[current_obs_index]["TIME"][0].astype("float") - myobs[0].obs_info["T_START"]
plt.vlines(mplet_start_time_ns*1e-9,.98,1.02,color="red",label="mplet start time",ls="--")
plt.legend()
plt.ticklabel_format(axis="x",style="sci",useOffset=False)
plt.show()

In [ ]:
from scipy.stats import expon
dt_run = np.diff(np.sort(phottable["TIME"]))
dt_mask = np.diff(np.sort(phottable[mask]["TIME"]))
params_run = expon.fit(dt_run)
params_mask = expon.fit(dt_mask)
print(f"params_run : {params_run}")
print(f"params_mask : {params_mask}")

In [ ]:
lambda_mask_fit = 1/params_mask[1]
rate_boe = len(phottable[mask])/myobs[0].obs_info["LIVETIME"]
print(lambda_mask_fit,rate_boe)

In [ ]:
# plt.hist(expon.rvs(*params_run,int(1e3)),histtype="step",label="rvs hist",density=True)
x = np.linspace(0,3,1000)
plt.plot(x,expon.pdf(x,*params_run))
plt.hist(dt_run,label="dt masked",density=True,histtype="step",color="blue")
plt.legend()
plt.show()

In [ ]:
rate_run = len(phottable)/myobs[0].obs_info["LIVETIME"]
rate_masked_run = len(phottable[mask])/myobs[0].obs_info["LIVETIME"]
print(rate_run,rate_masked_run)

## Analysing the navigation table

In [28]:
import dill
import numpy as np

with open("pickles/obs_id_to_index_per_ds_dict.pkl","rb") as f:
    navdict = dill.load(f)
with open("pickles/navigationtable.pkl","rb") as f:
    navtab = dill.load(f)

# with open("pickles/mplets.pkl", "rb") as f:
#     mplets = dill.load(f)

In [25]:
a = np.array([1,2],dtype=np.int64)
b = [1,2,3,3]
b[a[0]]

2

In [1]:
from scanner_interpretation import getDataStores
ds = getDataStores()


TypeError: 'numpy.int64' object is not iterable

In [3]:
type(ds[0].obs_ids)

numpy.ndarray

In [4]:
obs = ds[0].get_observations([ds[0].obs_ids[0]])
obs[0].obs_info

{'EXTNAME': 'EVENTS',
 'CREATOR': 'SASH FITS::EventListWriter',
 'TELESCOP': 'HESS',
 'ORIGIN': 'H.E.S.S. Collaboration',
 'INSTRUME': 'H.E.S.S. Phase I',
 'EV_CLASS': 'std_ImPACT',
 'OBS_ID': 18361,
 'TSTART': 96157130.0,
 'TSTOP': 96158011.0,
 'MJDREFI': 51910,
 'MJDREFF': 0.000742870370370241,
 'TIMEUNIT': 's',
 'TIMESYS': 'TT',
 'TIMEREF': 'local',
 'TASSIGN': 'Namibia',
 'TELAPSE': 0,
 'ONTIME': 881.0,
 'LIVETIME': 881.0,
 'DEADC': 1.0,
 'OBJECT': 'Crab Nebula',
 'RA_OBJ': 83.633333333333,
 'DEC_OBJ': 22.014444444444,
 'RA_PNT': 83.633333333333,
 'DEC_PNT': 22.514444444444,
 'GLON_PNT': 184.133314364185,
 'GLAT_PNT': -5.51637925194063,
 'ALT_PNT': 37.6661202492352,
 'AZ_PNT': 327.963142353231,
 'RADECSYS': 'FK5',
 'EQUINOX': 2000,
 'CONV_DEP': 0.0,
 'CONV_RA': 83.633333333333,
 'CONV_DEC': 22.014444444444,
 'OBS_MODE': 'WOBBLE',
 'N_TELS': 3,
 'TELLIST': '2,3,4',
 'GEOLAT': -23.2717777777778,
 'GEOLON': 16.5002222222222,
 'ALTITUDE': 1835.0,
 'MUONEFF': 0.0940089424451192,
 'MUONC